In [47]:
!pip install yfinance

In [48]:
import yfinance as yf
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [49]:
ticker="^NSEI"
data=yf.download(ticker,start="2018-01-01",end="2024-01-01")
df=pd.DataFrame(data)

[*********************100%%**********************]  1 of 1 completed


In [50]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,10477.549805,10495.200195,10404.650391,10442.200195,10442.200195,153400
2018-01-03,10482.650391,10503.599609,10429.549805,10443.200195,10443.200195,167300
2018-01-04,10469.400391,10513.000000,10441.450195,10504.799805,10504.799805,174900
2018-01-05,10534.250000,10566.099609,10520.099609,10558.849609,10558.849609,180900
2018-01-08,10591.700195,10631.200195,10588.549805,10623.599609,10623.599609,169000
...,...,...,...,...,...,...
2023-12-22,21295.849609,21390.500000,21232.449219,21349.400391,21349.400391,284700
2023-12-26,21365.199219,21477.150391,21329.449219,21441.349609,21441.349609,219500
2023-12-27,21497.650391,21675.750000,21495.800781,21654.750000,21654.750000,256500


a)

In [51]:
def ema(data,column,time_period):
    return data[column].ewm(span=time_period,adjust=False).mean()


In [52]:
df["ema12"]=ema(df,"Close",12)
df["ema26"]=ema(df,"Close",26)
df["macd"]=df["ema12"]-df["ema26"]
df["signal"]=ema(df,"macd",9)

In [53]:
buy_index1=[]
sell_index1=[]
signal=[0]*(len(df)-1)
for index in range(0,len(df)-1):
    if((df["macd"].iloc[index]<=df["signal"].iloc[index]) and (df["macd"].iloc[index+1]>df["signal"].iloc[index+1])):
        buy_index1.append(index)
        signal[index]=1
    elif((df["macd"].iloc[index]>=df["signal"].iloc[index]) and (df["macd"].iloc[index+1]<df["signal"].iloc[index+1])):
        sell_index1.append(index)
        signal[index]=-1

In [54]:
sell_index1.append(1476)
signal.append(-1)
df["returns_percentage"]=np.nan

In [55]:
buy_index1

[0,
 38,
 46,
 58,
 105,
 127,
 157,
 201,
 233,
 254,
 267,
 282,
 334,
 365,
 394,
 418,
 434,
 462,
 476,
 494,
 515,
 520,
 547,
 582,
 599,
 608,
 621,
 647,
 674,
 697,
 734,
 758,
 799,
 812,
 879,
 915,
 928,
 966,
 978,
 1031,
 1077,
 1102,
 1155,
 1178,
 1206,
 1243,
 1257,
 1276,
 1291,
 1329,
 1336,
 1352,
 1397,
 1423,
 1440]

In [56]:
sell_index1

[21,
 40,
 50,
 89,
 114,
 153,
 165,
 229,
 239,
 260,
 274,
 308,
 351,
 368,
 417,
 428,
 451,
 463,
 484,
 499,
 519,
 522,
 576,
 596,
 606,
 616,
 628,
 650,
 689,
 727,
 746,
 769,
 802,
 847,
 911,
 917,
 934,
 973,
 997,
 1052,
 1092,
 1140,
 1157,
 1201,
 1213,
 1249,
 1267,
 1279,
 1321,
 1333,
 1338,
 1369,
 1410,
 1428,
 1476]

In [57]:
for i in range(len(buy_index1)):
    df["returns_percentage"].iloc[i]=(df["Close"].iloc[sell_index1[i]]-df["Close"].iloc[buy_index1[i]])/df["Close"].iloc[buy_index1[i]]*100

In [58]:
mean1=df["returns_percentage"].sum()/len(buy_index1)
sum_of_squares1=0
for i in range(len(buy_index1)):
    sum_of_squares1=sum_of_squares1+df["returns_percentage"].iloc[i]*df["returns_percentage"].iloc[i]
std1=np.sqrt(sum_of_squares1/len(buy_index1)-mean1*mean1)
sharpe_ratio1=(np.sqrt(len(buy_index1))*mean1-10)/std1 #Risk Free Rate = 10%
sharpe_ratio1
initial_capital1=10000000
df["capital"]=np.nan
def capital():
    global initial_capital1
    for i in range(len(buy_index1)):
        no_of_shares=int(initial_capital1/df["Close"].iloc[buy_index1[i]])
        remaining_capital=initial_capital1-no_of_shares*df["Close"].iloc[buy_index1[i]]
        initial_capital1=no_of_shares*df["Close"].iloc[sell_index1[i]]+remaining_capital
        df["capital"].iloc[i]=initial_capital1
    return initial_capital1
print("Final Capital is:",capital())


Final Capital is: 48326056.6953125


In [59]:
sharpe_ratio1

2.950025464684698

In [60]:
df["returns_percentage"].head(56)

Date
2018-01-02     5.503631
2018-01-03    -0.909110
2018-01-04    -2.171013
2018-01-05     6.804131
2018-01-08     0.241466
2018-01-09     5.412778
2018-01-10     1.114550
2018-01-11     6.617151
2018-01-12    -0.347956
2018-01-15    -0.507953
2018-01-16    -0.571434
2018-01-17     7.222825
2018-01-18     4.374886
2018-01-19    -0.832468
2018-01-22    -0.779795
2018-01-23     4.390089
2018-01-24     4.245167
2018-01-25     0.416920
2018-01-29     0.746811
2018-01-30    -0.851613
2018-01-31    -0.512893
2018-02-01     0.736707
2018-02-02    10.333773
2018-02-05     8.690288
2018-02-06     2.187452
2018-02-07     3.572853
2018-02-08     1.638960
2018-02-09     1.528467
2018-02-12     4.496817
2018-02-14    15.552335
2018-02-15     3.230951
2018-02-16     2.224483
2018-02-19     1.030746
2018-02-20     7.605925
2018-02-21    10.701819
2018-02-22     0.180388
2018-02-23     1.034632
2018-02-26    -2.773656
2018-02-27     4.429905
2018-02-28     5.410867
2018-03-01    -0.395610
2018-03-05 

MaxDrawDown

In [61]:
df["drawdowns"]=np.nan
for i in range(0,len(buy_index1)-1):
    df["drawdowns"].iloc[i]=(-df["capital"].iloc[i+1]+df["capital"].iloc[i])/df["capital"].iloc[i]*100
print("MaxDrawDown is:",df["drawdowns"].min())



MaxDrawDown is: -15.544355917879102


b) Bollinger Bands Strategy

In [62]:
df["sma_20"]=df["Close"].rolling(window=20).mean()
df["upper_band"]=df["Close"].rolling(window=20).mean()+2*df["Close"].rolling(window=20).std()
df["lower_band"]=df["Close"].rolling(window=20).mean()-2*df["Close"].rolling(window=20).std()
buy_index2=[]
sell_index2=[]
signal2=[0]*len(df)

In [63]:

def bb():
    temp=0
    for i in range(0,len(df)-1):
        if(df["Close"].iloc[i]>df["lower_band"].iloc[i] and df["Close"].iloc[i+1]<=df["lower_band"].iloc[i+1] and temp==0):
            buy_index2.append(i+1)
            signal2[i+1]=1
            temp=1
        if(df["Close"].iloc[i]<df["upper_band"].iloc[i] and df["Close"].iloc[i+1]>=df["upper_band"].iloc[i+1] and temp==1):
            sell_index2.append(i+1)
            signal2[i+1]=-1
            temp=0
bb()

In [64]:
buy_index2

[42, 172, 326, 509, 667, 755, 954, 1021, 1165, 1224, 1431]

In [65]:
sell_index2

[129, 223, 419, 586, 698, 868, 985, 1112, 1188, 1298, 1458]

In [66]:
df["returns_percentage2"]=np.nan
for i in range(len(buy_index2)):
    df["returns_percentage2"].iloc[i]=(df["Close"].iloc[sell_index2[i]]-df["Close"].iloc[buy_index2[i]])/df["Close"].iloc[buy_index2[i]]*100


In [67]:
df["returns_percentage2"]

Date
2018-01-02     6.810254
2018-01-03    -3.798891
2018-01-04    -1.945574
2018-01-05   -16.072480
2018-01-08     8.666664
                ...    
2023-12-22          NaN
2023-12-26          NaN
2023-12-27          NaN
2023-12-28          NaN
2023-12-29          NaN
Name: returns_percentage2, Length: 1477, dtype: float64

In [68]:
mean2=df["returns_percentage2"].sum()/len(buy_index2)
sum_of_squares2=0
for i in range(len(buy_index2)):
    sum_of_squares2=sum_of_squares2+df["returns_percentage2"].iloc[i]*df["returns_percentage2"].iloc[i]
std2=np.sqrt(sum_of_squares2/len(buy_index2)-mean2*mean2)
sharpe_ratio2=(np.sqrt(len(buy_index2))*mean2-10)/std2 #Risk Free Rate = 10%
initial_capital2=10000000
df["capital2"]=np.nan
def capital3():
    global initial_capital2
    for i in range(len(buy_index2)):
        no_of_shares2=int(initial_capital2/df["Close"].iloc[buy_index2[i]])
        remaining_capital2=initial_capital2-no_of_shares2*df["Close"].iloc[buy_index2[i]]
        initial_capital2=no_of_shares2*df["Close"].iloc[sell_index2[i]]+remaining_capital2
        df["capital2"].iloc[i]=initial_capital2
    return initial_capital2
print("Final Capital is:",capital3())

Final Capital is: 12110507.981445312


In [69]:
sharpe_ratio2

-0.37469137849652395

In [70]:
df["drawdowns2"]=np.nan
for i in range(0,len(buy_index2)-1):
    df["drawdowns2"].iloc[i]=(-df["capital2"].iloc[i+1]+df["capital2"].iloc[i])/df["capital2"].iloc[i]*100
print("MaxDrawDown is:",df["drawdowns2"].min())


MaxDrawDown is: -16.767960638466448


e)Heiken Aishi Candles

In [71]:
df2=pd.DataFrame()
df2["Close"]=0.25*(df["Close"]+df["Open"]+df["High"]+df["Low"])
df2["Open"]=0
df2["High"]=0
df2["Low"]=0
x=len(df)
def max(a,b):
    if(a>=b):
        return a
    else:
        return b
def min(a,b):
    if(a<=b):
        return a
    else:
        return b
df2["Open"].iloc[0]=df2["Close"].iloc[0]
for index in range(1,x):
        df2["Open"].iloc[index]=0.5*(df2["Close"].iloc[index-1]+df2["Open"].iloc[index-1])
for index in range(0,x):   
     df2["High"].iloc[index]=max(max(df2["Close"].iloc[index],df2["Open"].iloc[index]),df["High"].iloc[index])
     df2["Low"][index]=min(min(df2["Close"].iloc[index],df2["Open"].iloc[index]),df["Low"].iloc[index])
df2["Volume"]=df["Volume"]

In [72]:
df

,Open,High,Low,Close,Adj Close,Volume,ema12,ema26,macd,signal,returns_percentage,capital,drawdowns,sma_20,upper_band,lower_band,returns_percentage2,capital2,drawdowns2
Date,,,,,,,,,,,,,,,,,,,
2018-01-02,10477.549805,10495.200195,10404.650391,10442.200195,10442.200195,153400,10442.200195,10442.200195,0.000000,0.000000,5.503631,1.054999e+07,0.908572,NaN,NaN,NaN,6.810254,1.068055e+07,3.797975
2018-01-03,10482.650391,10503.599609,10429.549805,10443.200195,10443.200195,167300,10442.354041,10442.274269,0.079772,0.015954,-0.909110,1.045413e+07,2.170708,NaN,NaN,NaN,-3.798891,1.027491e+07,1.944196
2018-01-04,10469.400391,10513.000000,10441.450195,10504.799805,10504.799805,174900,10451.961082,10446.905791,5.055291,1.023822,-2.171013,1.022721e+07,-6.802631,NaN,NaN,NaN,-1.945574,1.007514e+07,16.062356
2018-01-05,10534.250000,10566.099609,10520.099609,10558.849609,10558.849609,180900,10468.405471,10455.197925,13.207546,3.460567,6.804131,1.092292e+07,-0.241395,NaN,NaN,NaN,-16.072480,8.456836e+06,-8.664236
2018-01-08,10591.700195,10631.200195,10588.549805,10623.599609,10623.599609,169000,10492.281492,10467.672124,24.609368,7.690327,0.241466,1.094929e+07,-5.410662,NaN,NaN,NaN,8.666664,9.189556e+06,-16.767961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,21295.849609,21390.500000,21232.449219,21349.400391,21349.400391,284700,21109.357460,20702.507591,406.849869,400.391457,NaN,NaN,NaN,20831.387500,21877.731494,19785.043506,NaN,NaN,NaN
2023-12-26,21365.199219,21477.150391,21329.449219,21441.349609,21441.349609,219500,21160.433176,20757.236629,403.196546,400.952475,NaN,NaN,NaN,20913.720020,21872.033026,19955.407013,NaN,NaN,NaN
2023-12-27,21497.650391,21675.750000,21495.800781,21654.750000,21654.750000,256500,21236.481918,20823.719101,412.762817,403.314543,NaN,NaN,NaN,21001.972559,21885.382787,20118.562330,NaN,NaN,NaN


In [73]:
df2["ema12"]=ema(df2,"Close",12)
df2["ema26"]=ema(df2,"Close",26)
df2["macd"]=df2["ema12"]-df2["ema26"]
df2["signal"]=ema(df2,"macd",9)

In [74]:
buy_index3=[]
sell_index3=[]
signal3=[0]*(len(df2)-1)
for index in range(0,len(df2)-1):
    if((df2["macd"].iloc[index]<=df2["signal"].iloc[index]) and (df2["macd"].iloc[index+1]>df2["signal"].iloc[index+1])):
        buy_index3.append(index)
        signal3[index]=1
    elif((df2["macd"].iloc[index]>=df2["signal"].iloc[index]) and (df2["macd"].iloc[index+1]<df2["signal"].iloc[index+1])):
        sell_index3.append(index)
        signal3[index]=-1

In [75]:
sell_index3.append(1476)
signal2.append(-1)
df2["returns_percentage"]=np.nan
for i in range(len(buy_index3)):
    df2["returns_percentage"].iloc[i]=(df2["Close"].iloc[sell_index3[i]]-df2["Close"].iloc[buy_index3[i]])/df2["Close"].iloc[buy_index3[i]]*100

In [76]:
mean3=df2["returns_percentage"].sum()/len(buy_index3)   #HEIKEN AISHI METHOD FOR MACD
sum_of_squares3=0
for i in range(len(buy_index3)):
    sum_of_squares3=sum_of_squares3+df2["returns_percentage"].iloc[i]*df2["returns_percentage"].iloc[i]
std3=np.sqrt(sum_of_squares3/len(buy_index3)-mean3*mean3)
sharpe_ratio3=(np.sqrt(len(buy_index3))*mean3-10)/std3 #Risk Free Rate = 10%
initial_capital3=10000000
df2["capital"]=np.nan
def capital2():
    global initial_capital3
    for i in range(len(buy_index3)):
        no_of_shares3=int(initial_capital3/df2["Close"].iloc[buy_index3[i]])
        remaining_capital3=initial_capital3-no_of_shares3*df2["Close"].iloc[buy_index3[i]]
        initial_capital3=no_of_shares3*df2["Close"].iloc[sell_index3[i]]+remaining_capital3
        df2["capital"].iloc[i]=initial_capital3
    return initial_capital3
print("Final Capital is:",capital2())


Final Capital is: 45792550.00512695


In [77]:
sharpe_ratio3

3.087828268582717

In [78]:
df2["drawdowns"]=np.nan
for i in range(0,len(buy_index3)-1):
    df2["drawdowns"].iloc[i]=(-df2["capital"].iloc[i+1]+df2["capital"].iloc[i])/df2["capital"].iloc[i]*100
print("MaxDrawDown is:",df2["drawdowns"].min())


MaxDrawDown is: -13.676128985267649


In [79]:
#HEIKEN AISHI METHOD FOR BB
df2["sma_20"]=df2["Close"].rolling(window=20).mean()
df2["upper_band"]=df2["Close"].rolling(window=20).mean()+2*df2["Close"].rolling(window=20).std()
df2["lower_band"]=df2["Close"].rolling(window=20).mean()-2*df2["Close"].rolling(window=20).std()
buy_index4=[]
sell_index4=[]
signal4=[0]*len(df)

def bb2():
    temp2=0
    for i in range(0,len(df2)-1):
        if(df2["Close"].iloc[i]>df2["lower_band"].iloc[i] and df2["Close"].iloc[i+1]<=df2["lower_band"].iloc[i+1] and temp2==0):
            buy_index4.append(i+1)
            signal4[i+1]=1
            temp2=1
        if(df["Close"].iloc[i]<df["upper_band"].iloc[i] and df["Close"].iloc[i+1]>=df["upper_band"].iloc[i+1] and temp2==1):
            sell_index4.append(i+1)
            signal4[i+1]=-1
            temp2=0
bb2()

In [80]:
buy_index4

[24, 178, 264, 327, 509, 667, 754, 954, 1021, 1165, 1225, 1432]

In [81]:
sell_index4

[129, 223, 270, 419, 586, 698, 868, 985, 1112, 1188, 1298, 1458]

In [82]:
df2["returns_percentage2"]=np.nan
for i in range(len(buy_index4)):
    df2["returns_percentage2"].iloc[i]=(df2["Close"].iloc[sell_index4[i]]-df2["Close"].iloc[buy_index4[i]])/df2["Close"].iloc[buy_index4[i]]*100

In [83]:
df2["returns_percentage2"]

Date
2018-01-02     4.847608
2018-01-03    -2.901452
2018-01-04     3.482745
2018-01-05    -3.438798
2018-01-08   -16.077513
                ...    
2023-12-22          NaN
2023-12-26          NaN
2023-12-27          NaN
2023-12-28          NaN
2023-12-29          NaN
Name: returns_percentage2, Length: 1477, dtype: float64

In [84]:
mean4=df2["returns_percentage2"].sum()/len(buy_index4)
sum_of_squares4=0
for i in range(len(buy_index4)):
    sum_of_squares4=sum_of_squares4+df2["returns_percentage2"].iloc[i]*df2["returns_percentage2"].iloc[i]
std4=np.sqrt(sum_of_squares4/len(buy_index4)-mean4*mean4)
sharpe_ratio4=(np.sqrt(len(buy_index4))*mean4-10)/std4 #Risk Free Rate = 10%
initial_capital4=10000000
df2["capital2"]=np.nan
def capital4():
    global initial_capital4
    for i in range(len(buy_index4)):
        no_of_shares4=int(initial_capital4/df2["Close"].iloc[buy_index4[i]])
        remaining_capital4=initial_capital4-no_of_shares4*df2["Close"].iloc[buy_index4[i]]
        initial_capital4=no_of_shares4*df2["Close"].iloc[sell_index4[i]]+remaining_capital4
        df2["capital2"].iloc[i]=initial_capital4
    return initial_capital4
print("Final Capital is:",capital4())

Final Capital is: 11598622.497558594


In [85]:
sharpe_ratio4

-0.6318094227354509

In [86]:
df2["drawdowns2"]=np.nan
for i in range(0,len(buy_index4)-1):
    df2["drawdowns2"].iloc[i]=(-df2["capital2"].iloc[i+1]+df2["capital2"].iloc[i])/df2["capital2"].iloc[i]*100
print("MaxDrawDown is:",df2["drawdowns2"].min())

MaxDrawDown is: -15.130819745564942


If we Compare the Parameters we get More Profit in Denoised Data than Actual Data for BB and for MACD we get Actual Data gives More Profit than Denoised Data

c)

In [87]:
buy_index5=[]   #STOP LOSS STRATEGY FOR MACD
for i in range(len(buy_index1)):
    buy_index5.append(buy_index1[i])
sell_index5=[]
def strategy(stop_loss):
    k=0
    for i in range(len(df)):
        if(signal[i]==1):
            for j in range(i+1,len(df)):
                if((df["Close"].iloc[j]<=(1-stop_loss/100)*df["Close"].iloc[k]) or (signal[j]==-1) and k<=len(buy_index1)):
                    sell_index5.append(j)
                    k=k+1
                    break

strategy(5)
    

In [88]:
buy_index5

[0,
 38,
 46,
 58,
 105,
 127,
 157,
 201,
 233,
 254,
 267,
 282,
 334,
 365,
 394,
 418,
 434,
 462,
 476,
 494,
 515,
 520,
 547,
 582,
 599,
 608,
 621,
 647,
 674,
 697,
 734,
 758,
 799,
 812,
 879,
 915,
 928,
 966,
 978,
 1031,
 1077,
 1102,
 1155,
 1178,
 1206,
 1243,
 1257,
 1276,
 1291,
 1329,
 1336,
 1352,
 1397,
 1423,
 1440]

In [89]:
sell_index5

[21,
 40,
 50,
 89,
 114,
 153,
 165,
 229,
 239,
 260,
 274,
 308,
 351,
 368,
 417,
 428,
 451,
 463,
 484,
 499,
 519,
 522,
 548,
 583,
 606,
 616,
 628,
 650,
 689,
 727,
 746,
 769,
 802,
 847,
 911,
 917,
 934,
 973,
 997,
 1052,
 1092,
 1140,
 1157,
 1201,
 1213,
 1249,
 1267,
 1279,
 1321,
 1333,
 1338,
 1369,
 1410,
 1428,
 1476]

In [90]:
buy_index6=[] #STOP LOSS STRATEGY FOR BB
for i in range(len(buy_index2)):
    buy_index6.append(buy_index2[i])
sell_index6=[]
def strategy2(stop_loss):
    y=0
    for i in range(len(df)):
        if(signal2[i]==1):
            for j in range(i+1,len(df)):
                if((df["Close"].iloc[j]<=(1-stop_loss/100)*df["Close"].iloc[y]) or (signal[j]==-1) and y<=len(buy_index2)):
                    sell_index6.append(j)
                    y=y+1
                    break

strategy2(5)

In [91]:
buy_index6

[42, 172, 326, 509, 667, 755, 954, 1021, 1165, 1224, 1431]

In [92]:
sell_index6

[50, 229, 351, 519, 689, 769, 973, 1052, 1201, 1249, 1476]

Since we got the indices of buy and sell after considering stop loss we can find the required parameters similar to previous parts

d)Trailing StopLoss